In [1]:
import pandas as pd
import editdistance
import geopy.distance as dst

## Read new and old pandas dataframe

In [2]:
df = pd.read_csv('/code/gocode/src/geosure/mdf20180806.tsv', encoding='utf-16',sep='\t')
df_na = pd.read_csv('/Users/jorge/Desktop/mdf_na_new.csv', encoding='latin1')
df_na.columns = [r.lower() for r in df_na.columns]
# df_tx = pd.read_csv('/Users/jorge/Desktop/mdf_texas.csv')

print(df.shape)
print(df_na.shape)
# df_na = pd.read_csv('/Users/jorge/Desktop/na_mdf.csv')

(23471, 25)
(4159, 23)


## Organized needed columns

In [3]:
old_columns = df.columns[:-7].tolist()
new_columns = df.columns[-7:].tolist()

print(old_columns)
print('----------')
print(new_columns)

comp_columns = ['wiki_city', 'country', 'city_original', 'district', 'coords']

['geosure_id', 'record', 'continent', 'wiki_continent', 'country', 'wiki_country', 'country_code', 'province', 'wiki_city', 'city', 'city_original', 'city_type', 'district', 'language', 'language_secondary', 'population', 'latitude', 'longitude']
----------
['composite', 'physical', 'women', 'theft', 'political', 'health', 'lgbtq']


In [4]:
df_na_old = df.loc[(df['continent'] == 'North America')].copy().reset_index(drop=True)
print(df_na_old.shape)
df_na_old.head()

(1020, 25)


,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
0,G2000000,1,North America,Q49,Bermuda,Q23635,BM,Ontario,Q30985,Hamilton,...,1010.0,32.294816,-64.781375,22,18,17,23,25,30,50
1,G2000001,2,North America,Q49,Canada,Q16,CA,Alberta,Q2096,Edmonton,...,812201.0,53.544389,-113.490927,24,29,25,27,19,25,50
2,G2000002,3,North America,Q49,Canada,Q16,CA,Alberta,Q36312,Calgary,...,1096833.0,51.045325,-114.058101,24,27,23,26,19,25,50
3,G2000003,4,North America,Q49,Canada,Q16,CA,British Columbia,Q2132,Victoria,...,80017.0,48.428421,-123.365644,29,36,34,38,19,28,50
4,G2000004,5,North America,Q49,Canada,Q16,CA,British Columbia,Q24639,Vancouver,...,603502.0,49.261226,-123.113927,24,27,24,27,19,25,50


### Use coordinates

In [5]:
def create_coords(df_old, df_new):
    df_old['coords'] = list(zip(df_old['latitude'], df_old['longitude']))
    df_new['coords'] = list(zip(df_new['latitude'], df_new['longitude']))
    
    return df_old, df_new


df_na_old, df_na = create_coords(df_na_old, df_na)

In [6]:
def calc_word_dist(old, new):    
    return editdistance.eval(old, new), old, new


def compute_distance(data, item, df_old):
    if item == 'coords':
        return df_old['coords'].apply(lambda x: dst.vincenty(data[item], x).km)
    tuple_dists = df_old[item].apply(lambda x: calc_word_dist(x, data[item]))
    new_col_names = ['{0}_{1}'.format(c, item) for c in ['dist', 'old', 'new']]

    df = pd.DataFrame(tuple_dists.values.tolist(), columns=new_col_names)

    if item == 'wiki_city':
        df['old_index'] = df_old.index.tolist()
        df['new_index'] = data.name
    return df


def compute_distances(data, columns, df_old):
    distances = [compute_distance(data, col, df_old) for col in columns]
    df = pd.concat(distances, axis=1)
    df.sort_values(by=['coords', 'dist_wiki_city', 'dist_country', 'dist_district'], inplace=True)
    
    return df.iloc[:5]


def normalize_values(df):
    for item in ['dist_wiki_city', 'dist_city_original', 'dist_district', 'dist_country', 'coords']:
        df[item] = df[item] / df[item].max()

    return df


def create_costs(df_new, df_old, comp_columns):
    distances = [compute_distances(r, comp_columns, df_old) for k, r in df_new.iterrows()]
    df = pd.concat(distances, axis=0)
    df = df[['new_index', 'old_index', 'dist_wiki_city', 'old_wiki_city', 'new_wiki_city',
             'dist_country', 'old_country', 'new_country', 'dist_city_original', 'old_city_original',
             'new_city_original', 'dist_district', 'old_district', 'new_district', 'coords']]
    df = normalize_values(df)
    
    return df


df_costs = create_costs(df_na, df_na_old, comp_columns)
df_cost = df_costs.reset_index(drop=True)

In [7]:
df_cost.head()
# PLAY WITH THE COST DATAFRAME HERE

,new_index,old_index,dist_wiki_city,old_wiki_city,new_wiki_city,dist_country,old_country,new_country,dist_city_original,old_city_original,new_city_original,dist_district,old_district,new_district,coords
0,0,0,0.000,Q30985,Q30985,0.333333,Bermuda,Bermuda (UK),0.000000,Hamilton,Hamilton,0.0,Overall,Overall,0.000000
1,0,381,0.500,Q49259,Q30985,0.800000,United States,Bermuda (UK),0.684211,Virginia Beach,Hamilton,0.0,Overall,Overall,0.135214
2,0,379,0.625,Q49222,Q30985,0.800000,United States,Bermuda (UK),0.526316,Chesapeake,Hamilton,0.0,Overall,Overall,0.137810
3,0,382,0.625,Q49231,Q30985,0.800000,United States,Bermuda (UK),0.421053,Norfolk,Hamilton,0.0,Overall,Overall,0.138200
4,0,384,0.625,Q342043,Q30985,0.800000,United States,Bermuda (UK),0.105263,Hampton,Hamilton,0.0,Overall,Overall,0.139679


## Create cost matrix

In [59]:
def filter_df(df_cost):
    df_cost['cost'] = df_cost[['dist_wiki_city', 'dist_city_original', 'dist_district', 'coords']].sum(axis=1)
    df_reduced = df_cost.iloc[:-1:5].loc[df_cost['cost'] <= 0.05]
  
    return df_reduced


def get_ids(df_reduced):
    return list(df_reduced[['old_index', 'new_index']].T.to_dict().values())


def create_row(df_old, df_new, ids_dict):
    res_dict = df_old.iloc[ids_dict['old_index']][old_columns].to_dict()
    res_dict.update(df_new.iloc[ids_dict['new_index']][new_columns].to_dict())

    return res_dict


def create_replace_df(df_old, df_new, df_cost):
    df_reduced = filter_df(df_cost)

    replace_list = [create_row(df_na_old, df_na, item) for item in get_ids(df_reduced)]
    df = pd.DataFrame(replace_list)
    df = df[old_columns + new_columns]
    
    return df


df_replace = create_replace_df(df_na_old, df_na, df_cost)
print(df_replace.shape)
df_replace.head()


(1006, 25)


,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
0,G2000000,1,North America,Q49,Bermuda,Q23635,BM,Ontario,Q30985,Hamilton,...,1010.0,32.294816,-64.781375,22,18,17,23,25,30,24
1,G2000746,747,North America,Q49,United States,Q30,US,California,Q65,Los Angeles,...,NaN,34.136572,-118.294211,42,37,36,37,41,57,47
2,G2000626,627,North America,Q49,United States,Q30,US,California,Q62,San Francisco,...,NaN,37.794256,-122.407153,17,17,17,20,13,12,23
3,G2000002,3,North America,Q49,Canada,Q16,CA,Alberta,Q36312,Calgary,...,1096833.0,51.045325,-114.058101,26,27,23,26,19,25,33
4,G2000818,819,North America,Q49,United States,Q30,US,California,Q65,Los Angeles,...,NaN,33.993823,-117.969632,27,19,24,18,35,48,29


## Drop items from dataframe. 

## Include new scores from removed

In [60]:
def replace_ids(df_replace, df_global):
    list_ids = df_replace['geosure_id'].tolist()
    print("Len geosure ids: {}".format(len(list_ids)))

    df = df_global.loc[~df_global['geosure_id'].isin(list_ids)].copy()
    print('New shape {0}'.format(df.shape))

    df = pd.concat([df, df_replace], axis=0).reset_index(drop=True)
    df.sort_values(by='geosure_id', inplace=True)

    return df

df_new = replace_ids(df_replace, df)
print('New shape after concat {0}'.format(df_new.shape))

Len geosure ids: 1006
New shape (22465, 25)
New shape after concat (23471, 25)


### Compute difference between scores

In [61]:
def check_diff(df, df_new):
    diffs = (df[new_columns].as_matrix().sum(axis=1) - df_new[new_columns].as_matrix().sum(axis=1))
    return (diffs != 0).sum()
    
print("Number of rows changed: {0}".format(check_diff(df, df_new)))

Number of rows changed: 992


## Include new items from NA

In [67]:
def get_counter(df_global):
    # Get geosure id counter.
    gs_id = df_global.loc[df_global['continent'] == 'North America'].iloc[-1]['geosure_id']
    int_id = int(gs_id.replace('G', '')) + 1
    
    return int_id


def include_code(df_add, df_global):
    # Get Country codes.
    countries = df_global.groupby(['country_code', 'country']).size().reset_index(name='count')
    countries.drop(labels=['count'], inplace=True, axis=1)

    df_add = df_add.merge(countries, how='inner', on='country')

    return df_add

def create_df_add(df_cost, df_new, df_global):
    ids_to_add = df_cost.iloc[:-1:5].loc[df_cost['cost'] > 0.05]['new_index']
    df_add = df_new.ix[ids_to_add]

    int_id = get_counter(df_global)

    df_add['geosure_id'] = pd.Series(['G' + str(x) for x in range(int_id, int_id + df_add.shape[0])],
                                      index=df_add.index)
    df_add.drop(columns=['coords'], inplace=True)

    df_add = include_code(df_add, df_global)
    df_add = df_add[old_columns + new_columns]

    return df_add

df_add = create_df_add(df_cost, df_na, df)
print(df_add.shape)
df_add.head()

(3153, 25)


,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
0,G2001020,74,North America,Q49,United States,Q30,US,Arizona,Q49261,Mesa,...,NaN,33.440007,-111.704826,38,39,32,40,33,40,45
1,G2001021,75,North America,Q49,United States,Q30,US,Arizona,Q49261,Mesa,...,NaN,33.442776,-111.713397,38,39,32,40,33,40,45
2,G2001022,76,North America,Q49,United States,Q30,US,Arizona,Q49261,Mesa,...,NaN,33.456023,-111.704826,36,36,31,38,32,39,42
3,G2001023,77,North America,Q49,United States,Q30,US,Arizona,Q49261,Mesa,...,NaN,33.356825,-111.663091,28,25,26,28,29,33,31
4,G2001024,78,North America,Q49,United States,Q30,US,Arizona,Q49261,Mesa,...,NaN,33.487634,-111.674270,21,15,22,19,26,29,20


### Concatenate with last dataframe.

In [68]:
df_final = pd.concat([df_new, df_add], axis=0)
df_final.sort_values(by='geosure_id', inplace=True)
print(df_final.shape)

(26624, 25)


# Update: Fix houston and Dallas

In [72]:
df = df_final.copy()
df_tx = pd.read_csv('/Users/jorge/Desktop/mdf_texas.csv')
print(df.shape)
print(df_tx.shape)

(26624, 25)
(151, 23)


In [75]:
# df_tx_old = df.loc[(df['city'].isin(['Houston', 'Dallas'])) & (df['country'] == 'United States')].copy().reset_index(drop=True)
df_tx_old = df.loc[(df['city'].isin(['Houston', 'Dallas']))].copy().reset_index(drop=True)
print(df_tx_old.shape)

(145, 25)


In [76]:
df_tx_old, df_tx = create_coords(df_tx_old, df_tx)

In [80]:
df_costs = create_costs(df_tx, df_tx_old, comp_columns)
df_cost = df_costs.reset_index(drop=True)
df_cost.shape

(755, 15)

In [83]:
df_replace = create_replace_df(df_tx_old, df_tx, df_cost)
print(df_replace.shape)
df_replace.head()

(142, 25)


,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
0,G2000001,2,North America,Q49,Canada,Q16,CA,Alberta,Q2096,Edmonton,...,812201.0,53.544389,-113.490927,22,18,17,23,25,30,24
1,G2000028,29,North America,Q49,Canada,Q16,CA,Ontario,Q200166,Kitchener,...,219153.0,43.434311,-80.477747,40,41,32,42,34,41,47
2,G2000137,138,North America,Q49,United States,Q30,US,California,Q494711,Lancaster,...,161963.0,34.686785,-118.154163,24,23,20,23,19,27,29
3,G2000091,92,North America,Q49,United States,Q30,US,California,Q49247,Anaheim,...,353504.0,33.835293,-117.914504,26,27,23,26,19,25,33
4,G2000055,56,North America,Q49,Canada,Q16,CA,Saskatchewan,Q2123,Regina,...,193100.0,50.454722,-104.606667,16,8,19,13,24,26,11


In [85]:
df_new = replace_ids(df_replace, df)
print('New shape after concat {0}'.format(df_new.shape))
print("Number of rows changed: {0}".format(check_diff(df, df_new)))

Len geosure ids: 142
New shape (26482, 25)
New shape after concat (26624, 25)
Number of rows changed: 141


In [88]:
df_add = create_df_add(df_cost, df_tx, df)
print(df_add.shape)
df_add.head()

(9, 25)


,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
0,G2004173,3618,North America,Q49,United States,Q30,US,Texas,Q16557,Dallas,...,NaN,32.698809,-96.832329,58,65,47,63,47,56,68
1,G2004174,3619,North America,Q49,United States,Q30,US,Texas,Q16557,Dallas,...,NaN,32.819581,-96.820470,42,45,35,54,35,45,35
2,G2004175,3624,North America,Q49,United States,Q30,US,Texas,Q16557,Dallas,...,NaN,32.786907,-96.896889,55,50,57,54,56,62,52
3,G2004176,3626,North America,Q49,United States,Q30,US,Texas,Q16557,Dallas,...,NaN,32.698809,-96.832329,58,65,47,63,47,56,68
4,G2004177,3649,North America,Q49,United States,Q30,US,Texas,Q16557,Dallas,...,NaN,32.776818,-96.711117,58,65,47,63,47,56,68


In [90]:
df_final = pd.concat([df_new, df_add], axis=0)
df_final.sort_values(by='geosure_id', inplace=True)
print(df_final.shape)

(26633, 25)


In [94]:
df_final.loc[(df_final['city'] == 'Houston') & (df_final['district'] == 'Overall')]

,geosure_id,record,continent,wiki_continent,country,wiki_country,country_code,province,wiki_city,city,...,population,latitude,longitude,composite,physical,women,theft,political,health,lgbtq
11167,G2000340,341,North America,Q49,United States,Q30,US,Texas,Q16555,Houston,...,2334348.0,29.760193,-95.36939,39,41,32,41,33,40,46


# Saving results

In [96]:
df_final.to_csv('/code/gocode/src/geosure/mdf20180827.tsv', index=False, sep='\t', encoding='utf-16')